In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from catboost import CatBoostClassifier
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../../'))

from core.train_utils import calculate_roi, get_winner_favorite, combine_df

In [4]:
df = pd.read_pickle('../../data/Catboost_v0/df_combined_total.pkl')
df['winner'] = df['winnerId'] == df['fighterId_1']
print(df.shape)

df = df[(df['f1_count_of_fights'] >= 2) & (df['f2_count_of_fights'] >= 2)]

train = df[(df['eventDate.date'].dt.year > 2005) & (df['eventDate.date'].dt.year < 2020)]

val  = df[df['eventDate.date'].dt.year >= 2020]
val = val[~val['odds_difference'].isna()]
val = val.reset_index()

df.shape, train.shape, val.shape

FileNotFoundError: [Errno 2] No such file or directory: '../../data/Catboost_v0/df_combined_total.pkl'

In [ ]:
train_cols = ['cumsumwinner_difference', 'cumsum_hitsTotal_difference', 'cumsum_hitsSuccessful_difference', 
              'cumsum_takedownTotal_difference', 'cumsum_takedownSuccessful_difference', 
              'cumsum_submissionAttempts_difference', 'cumsum_takeovers_difference', 
              'cumsum_accentedHitsTotal_difference', 'cumsum_accentedHitsSuccessful_difference', 
              'cumsum_knockdowns_difference', 'cumsum_protectionPassage_difference', 
              'cumsum_hitsHeadTotal_difference', 'cumsum_hitsHeadSuccessful_difference', 
              'cumsum_hitsBodyTotal_difference', 'cumsum_hitsBodySuccessful_difference', 
              'cumsum_hitsLegsTotal_difference', 'cumsum_hitsLegsSuccessful_difference', 
              'cumsum_accentedHitsPositionDistanceTotal_difference', 
              'cumsum_accentedHitsPositionDistanceSuccessful_difference', 
              'cumsum_accentedHitsPositionClinchTotal_difference', 
              'cumsum_accentedHitsPositionClinchSuccessful_difference', 
              'cumsum_accentedHitsPositionParterTotal_difference', 
              'cumsum_accentedHitsPositionParterSuccessful_difference', 
              'cumsumwinMethods_[DEC]_difference', 'cumsumwinMethods_[DQ]_difference', 
              'cumsumwinMethods_[KO]_difference', 'cumsumwinMethods_[SUB]_difference', 
              'count_of_fights_difference', 'age_difference'] #,  , 'odds_difference'

In [ ]:
X_train = train[train_cols]
y_train = train['winner'].astype(int)

X_val = val[train_cols]
y_val = val['winner'].astype(int)

print(train.shape, val.shape)
clf = CatBoostClassifier(iterations=4000, 
                           depth=3,
                           learning_rate=0.01,
                           loss_function='Logloss',
                           colsample_bylevel=1,
                           subsample=0.1,
                           random_strength=1,
#                            l2_leaf_reg=30,
#                            reg_lambda=40,
                           eval_metric='Accuracy',
                           boosting_type='Plain', # 'Plain'
                           bootstrap_type='Bernoulli',# 'Bernoulli' Bayesian
                           max_ctr_complexity=5,
                           task_type='GPU',
                          )

clf.fit(X_train, y_train, plot=True, eval_set=(X_val, y_val), verbose=False)

In [ ]:
clf.get_feature_importance(prettified=True)#['Feature Id'].values

In [ ]:
clf.save_model('../../models/Catboost_v0/catboost_v0_0_05.04.2021.cat')

In [ ]:
y_pred = clf.predict_proba(val[clf.feature_names_])
val['y_proba'] = y_pred[:,1]
val

In [ ]:
reversed_index = val[val['type'] == 'reversed'].index
y_proba_reversed = val[val['type'] == 'reversed']['y_proba'].apply(lambda x: 1-x)
y_proba_reversed 

In [ ]:
val = val[val['type'] == 'straight']
val['y_proba_reversed'] = y_proba_reversed
val['y_proba_mean'] = (np.array(y_proba_reversed) + np.array(val['y_proba'])) / 2
val['y_proba_mean']
val['y_pred'] = val['y_proba'] > 0.5

In [ ]:
print('Точность букмейкера')
val['bookmaker_bet'] = (val['f1_odds'] < val['f2_odds']).astype(int)
(val['bookmaker_bet'] == val['winner']).mean()

In [5]:
calculate_roi(val)

NameError: name 'val' is not defined

In [ ]:
val = get_winner_favorite(val)
bet_on_favorite = val[val['y_pred'] == val['favorite']]
calculate_roi(bet_on_favorite)

In [ ]:
bet_on_underdog = val[val['y_pred'] != val['favorite']]
print(bet_on_underdog.shape)
calculate_roi(bet_on_underdog)

In [ ]:
val.columns

In [6]:
calculate_roi??

In [7]:
val['cumsum_hitsHeadTotal_difference'].hist()

NameError: name 'val' is not defined

In [ ]:
df['c']